#### Example 1:
Let’s say we want to implement a counter to record how many time the word has been repeated. The first thing you may want to do is to define a dictionary in global scope, and then create a function to add in the words as key into this dictionary and also update the number of times it repeated. Below is the sample code:

In [1]:
counter = dict()

def count_word(word):  
    global counter  
    
    # some validation ...
    word = word.lower()
    if word not in ['the', 'or', 'and', 'is', 'am', 'are'] and not word.isdigit():
        counter[word] = counter.get(word, 0) + 1
        return counter[word]
    
    return 0

text = 'Java Python Java Java Python C# 1234 the'
for word in text.split():
    print(count_word(word))

1
1
2
3
2
1
0
0


In [4]:
counter

{'java': 3, 'python': 3, 'c#': 1}

In [3]:
count_word('python')

3

In [5]:
counter[1234] = 1

In [6]:
counter

{'java': 3, 'python': 3, 'c#': 1, 1234: 1}

In [ ]:
counter

# To make sure the count_word function updates the correct “counter”,
# we need to put the global keyword to explicitly tell Python interpreter
# to use the “counter” defined in global scope, not any variable we accidentally
# defined with the same name in the local scope (within this function).

#### issue1 :
The global variable is accessible to any of the other functions and you cannot guarantee your data won’t be modified by others. 

#### issue2 : 
The global variable exists in the memory as long as the program is still running, so you may not want to create so many global variables if not necessary.

In [7]:
#### Solution: bound the variable with the function inside another function

def word_counter():
    counter = dict()
    
    def count(word):
        nonlocal counter
        word = word.lower()
        if word not in ['the', 'or', 'and', 'is', 'am', 'are'] and not word.isdigit():
            counter[word] = counter.get(word, 0) + 1
            return counter[word]
        else:
            return 0
        
    return count

the counter dictionary is hidden from the public access and the functionality remains the same.

In [8]:
c = word_counter() # counter = dict() , count
d = word_counter() # # counter = dict() , count
#count
text = 'Java Python Java Java Python C# 1234 the'
for word in text.split():
    c(word)
    
text = 'Java Java Java C# 1234 the'
for word in text.split():
    d(word)

In [9]:
d('python')

1

Dont worry! It is a closure

-------

### Closures

Functions defined inside another function can access the outer (nonlocal) variables

In [ ]:
def outer():
    x = 'python' # non-local variable x (or free variable)
    def inner(): # a function defined inside the outer function 
        print(f'{x} rocks') # and referenced x, which is defined in the outer function
    return inner

when we consider inner, we really are looking at two things:
+ the function inner
+ the free variable x (with current value python)
  x is not in the local scope of inner, it is in somewhere else
  
This two things have to __bound together__ . It is called a closure.
closure is a nested function remembers and has access to variables in the scope of the function in which it is defined.

A Closure is a __function__ plus an extended scope that contains the __free variables__

In [10]:
def outer():
    x = 'python' # non-local variable x (or free variable)
    def inner(): # a function defined inside the outer function 
        print(f'{x} rocks') # and referenced x, which is defined in the outer function
    return inner

In [11]:
fn = outer() # fn is (inner + free variable). it creates inner (no run)

In [12]:
outer()

<function __main__.outer.<locals>.inner()>

In [13]:
fn() # 

python rocks


In [14]:
type(fn)

function

In [15]:
fn.__code__.co_freevars 

('x',)

As we can see, `x` is a free variable in the closure.

#### Why do we use closures?
1. Hide data with Python closure <br>
   Example 1
2. Convert small class to function with Python closure <br>
   Example 2
3. Decorators <br>
   Next part
    


#### Example 2
Occasionally in your project, you may want to implement a small utility class to do some simple task. Let’s take a look at the below example:

In [1]:
import requests

In [8]:
url = 'https://divar.ir/s/{city}/{item}?price={min_price}-{max_price}'.format(
    city = 'tehran', item = 'car', min_price = 100000000, max_price = 200000000)

In [10]:
print(url)

https://divar.ir/s/tehran/car?price=100000000-200000000


In [12]:
import requests

class RequestMaker:
    def __init__(self, base_url):
        self.url = base_url
        
    def request(self, **kwargs):
        return requests.get(self.url.format_map(kwargs))

In [13]:
divar = RequestMaker('https://divar.ir/s/{city}/{item}?price={min_price}-{max_price}')
divar.request(city = 'tehran', item = 'car', min_price = 100000000, max_price = 200000000)

<Response [200]>

In [14]:
divar.request(city = 'new york', item = 'car', min_price = 100000000, max_price = 200000000)

<Response [404]>

Since you’ve already seen in the word counter example, the closure can also hold the data for your later use, the above class can be converted into a function with closure:

In [15]:
import requests

def request_maker(url):
    def make_request(**kwargs):
        return requests.get(url.format_map(kwargs))
    return make_request


The code becomes more concise and achieves the same result. Take note that in the above code, we are able to pass in the arguments into the nested function with \*\*kwargs (or \*args).

In [16]:
divar = request_maker('https://divar.ir/s/{city}/{item}?price={min_price}-{max_price}')
divar(city = 'tehran', item = 'car', min_price = 100000000, max_price = 200000000)

<Response [200]>

### Multiple Instances of Closures

In [16]:
def pow(n):
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    return inner

In this example, `n`, in the function `inner` is a free variable, so we have a closure that contains `inner` and the free variable `n`

In [17]:
square = pow(2) # n= 2

In [18]:
square(6)

36

In [19]:
cube = pow(3) # n= 3

In [20]:
cube(5)

125